In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import numpy as np
from scipy import interpolate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from scipy.io import wavfile
from scipy.signal import decimate

## 0-1 watch file reading

In [ ]:
############## SMART WATCH ################################################

# List of CSV files
pkl_files = ['accelerometer', 'bvp', 'eda','temperature']
classes = ['Browsing on a Monitor','Hand Writing','Keyboard Typing','Reading Book',
           'Stretching','Talking in Phone','walking and talking to phone',
           'Walking Around','writing and talking to phone']

base_path = "D:/Download new/HAR"
base_path = "E:/Software/HAR_2023_dataset/OneDrive_2023/Synchronized HAR Data"

directories = glob.glob(os.path.join(base_path, "*-2023-*"))

client_set_watch = [ [] for i in range(35)]

# Loop through each matched directory
for directory in directories:
    class_set = []
    for class_name in classes:
        watch_modal = []
        for feature in pkl_files:
            # Define file path
            file_path = os.path.join(directory, class_name, "Smartwatch", "smartwatch", f"{feature}.pkl")
            # set the time stamp based on accelerometer 

            # Check if file exists
            if os.path.isfile(file_path):
                # Read each CSV file
                df = pd.read_pickle(file_path)

                # Convert DataFrame to numpy array and append to the list
                all_keys = list(df.keys())
                #making timestamps
                timestamps = np.arange(float(df['start_time_stamp']), float(df['start_time_stamp'] )
                                       + 1000*len(df[all_keys[0]])/df['sample_rate'], 1000/df['sample_rate'])
                df_values = np.array([df[key] for key in all_keys[:-2]])
                timestamps = timestamps[0:np.shape(df_values)[1]]
                df_values =  np.transpose(np.concatenate((np.expand_dims(timestamps,axis = 0), df_values), axis=0))
                watch_modal.append(df_values)
                #print(np.shape(df_values))
            else:
                #print(f"File {file_path} not found.")
                watch_modal.append(['not available'])
        class_set.append(watch_modal)
    client_set_watch[int(directory[-2:])-1] = class_set ##directory[-2:] retrieves the ID
    # Now 'class_set' is a list of numpy arrays for each class in the current directory
    # You can process it here before moving to the next directory

## 0-2 Phone file reading

In [ ]:
############# SMART PHONE ##################################

# List of CSV files
csv_files = ['Lacc', 'accel', 'gravity','gyro','mag','orientation','rotation']
classes = ['Browsing on a Monitor','Hand Writing','Keyboard Typing','Reading Book',
           'Stretching','Talking in Phone','walking and talking to phone',
           'Walking Around','writing and talking to phone']

# Base path
# Please enter the path directory for the dataset here
base_path = "E:/Software/HAR_2023_dataset/OneDrive_2023/Synchronized HAR Data"
# base_path = "H:/Arash/HAR/HAR_2023_dataset/2023 - FL - HAR/data/Synchronized HAR Data" # Office PC


# Get all directories within base_path that match the pattern
directories = glob.glob(os.path.join(base_path, "*-2023-*"))

client_set_phone = [ [] for i in range(35)]
# Loop through each matched directory
for directory in directories:
    class_set = []
    for class_name in classes:
        phone_modal = []
        for feature in csv_files:
            # Define file path
            file_path = os.path.join(directory, class_name, "Smartphone", f"{feature}.csv")
            
            # Check if file exists
            if os.path.isfile(file_path):
                # Read each CSV file
                df = pd.read_csv(file_path)

                # Convert DataFrame to numpy array and append to the list
                df = df.iloc[:, :-2]
                phone_modal.append(df.values)
            else:
                #print(f"File {file_path} not found.")
                phone_modal.append(['not available'])
        class_set.append(phone_modal)
    client_set_phone[int(directory[-2:])-1] = class_set ##directory[-2:] retrieves the ID
    # Now 'class_set' is a list of numpy arrays for each class in the current directory
    # You can process it here before moving to the next directory

## 0-3 Speaker file reading

In [ ]:
################# SMART SPEAKER ##########################################

down_rate = 4
# List of CSV files
csv_files = ['Lacc', 'accel', 'gravity','gyro','mag','orientation','rotation']
classes = ['Browsing on a Monitor','Hand Writing','Keyboard Typing','Reading Book',
           'Stretching','Talking in Phone','walking and talking to phone',
           'Walking Around','writing and talking to phone']

# Base path
base_path = "H:/Arash/HAR/HAR_2023_dataset/2023 - FL - HAR/data/Synchronized HAR Data" # Office PC
# base_path = "C:/NotWin/University/PhD/HAR/HAR_2023"
# base_path = "D:/OneDrive_2023-06-01(1)/2023 - FL - HAR/data/Synchronized HAR Data"
base_path = "E:/Software/HAR_2023_dataset/OneDrive_2023/Synchronized HAR Data"


# Get all directories within base_path that match the pattern
directories = glob.glob(os.path.join(base_path, "*-2023-*"))

client_set_speaker = [ [] for i in range(35)]
# Loop through each matched directory
for directory in directories:
    class_set = []
    for class_name in classes:
        speaker_modal = []
        for feature in csv_files:
            # Define file path
            file_path = os.path.join(directory, class_name, "SmartSpeaker", f"audio.wav")
             # Check if file exists
            if os.path.isfile(file_path):
                # Read the wav file (mono)
                sampling_rate, data = wavfile.read(file_path)

                # For stereo, data will be an array with two columns
                # If you want to convert it to mono
                if len(data.shape) > 1:
                    data = np.mean(data, axis=1)
                data = decimate(data, down_rate)
                speaker_modal.append(data)
            else:
                #print(f"File {file_path} not found.")
                speaker_modal.append(['not available'])
        class_set.append(speaker_modal)
    client_set_speaker[int(directory[-2:])-1] = class_set ##directory[-2:] retrieves the ID
    # Now 'class_set' is a list of numpy arrays for each class in the current directory
    # You can process it here before moving to the next directory
sampling_rate = int(sampling_rate/down_rate)

## 1-1 watch segmentation

In [ ]:
import numpy as np
from scipy import interpolate

def segment_data_watch(data, start_time, end_time, segment_size = 2, sampling_frequency = 64, overlap = 0.5):
    # mode 0: input is x,y,z
    # mode 0: input is x
    mode = 1
    if np.shape(data)[1] == 4:
        mode = 0

    timestamps = data[:,0]
    x = data[:, 1]
    if mode == 0:
        y = data[:, 2]
        z = data[:, 3]
    
    new_timestamps = np.arange(start_time, end_time, 1000/sampling_frequency)
    # Interpolate the data
    x_interpolated = interpolate.interp1d(timestamps, x,fill_value="extrapolate")(new_timestamps)
    if mode == 0:
        y_interpolated = interpolate.interp1d(timestamps, y,fill_value="extrapolate")(new_timestamps)
        z_interpolated = interpolate.interp1d(timestamps, z,fill_value="extrapolate")(new_timestamps)
    
    
    resampled_data = np.column_stack((new_timestamps, x_interpolated))
    if mode == 0:
        resampled_data = np.column_stack((new_timestamps, x_interpolated, y_interpolated, z_interpolated))  
    # Segment the data
    segments = []
    step_size = int(segment_size * sampling_frequency * (1 - overlap))
    for i in range(0, len(resampled_data) - segment_size * sampling_frequency + 1, step_size):
        segment = resampled_data[i:i + segment_size * sampling_frequency]
        segments.append(segment)
        
    # Convert list of segments to numpy array
    segments = np.array(segments)

    return segments

In [ ]:
################################## Preparing local datasets
OVERLAP = 0.2   # in precentage
Sample_FREQ = 50 # in Htz
seg_size = 2 #in second
clean_clients = [x for x in range(35) if x not in [0,2,5,6,17,27,30]]
local_datasets_watch = []
for client_idx in clean_clients:

    for class_id in range(9):
        watch_modal = client_set_watch[client_idx][class_id]
        for sig in range(4):
            if sig == 0:
                
                start_time = watch_modal[sig][:,0][0]
                end_time = watch_modal[sig][:,0][-1]
                x_data = segment_data_watch(watch_modal[sig],start_time = start_time,
                                        end_time = end_time, segment_size=seg_size, sampling_frequency=Sample_FREQ, overlap=OVERLAP)
                x_data = x_data[...,1:] #exluding the timestamps
                y_data = np.zeros([x_data.shape[0]]) + class_id  

            else:
                segments = segment_data_watch(watch_modal[sig],start_time = start_time,
                                        end_time = end_time, segment_size=seg_size, sampling_frequency=Sample_FREQ, overlap=OVERLAP)
                x_data = np.concatenate([x_data,segments[...,1:]],axis=-1)
        if class_id == 0:
            x_dataset = x_data
            y_dataset = y_data
            if client_idx == clean_clients[0]:
                x_central_watch = x_dataset
                y_central_watch = y_dataset
        else:
            x_dataset = np.concatenate([x_dataset,x_data],axis=0)
            y_dataset = np.concatenate([y_dataset,y_data],axis=0)
    local_datasets_watch.append( [x_dataset,y_dataset] )
    x_central_watch = np.concatenate((x_central_watch,x_dataset))
    y_central_watch = np.concatenate((y_central_watch,y_dataset))

## 1-2 Phone Segmentation

In [ ]:
def segment_data_phone(data, segment_size=2, sampling_frequency=1000, overlap=0.5):
    # Extract the columns
    timestamps = data[:, 0]
    x = data[:, 1]
    y = data[:, 2]
    z = data[:, 3]

    # Create a new time array with stable frequency
    new_timestamps = np.arange(timestamps[0], timestamps[-1], 1000/sampling_frequency)

    # Interpolate the data
    x_interpolated = interpolate.interp1d(timestamps, x)(new_timestamps)
    y_interpolated = interpolate.interp1d(timestamps, y)(new_timestamps)
    z_interpolated = interpolate.interp1d(timestamps, z)(new_timestamps)

    # Resampled data
    resampled_data = np.column_stack((new_timestamps, x_interpolated, y_interpolated, z_interpolated))

    # Segment the data
    segments = []
    step_size = int(segment_size * sampling_frequency * (1 - overlap))
    for i in range(0, len(resampled_data) - segment_size * sampling_frequency + 1, step_size):
        segment = resampled_data[i:i + segment_size * sampling_frequency]
        segments.append(segment)

    # Convert list of segments to numpy array
    segments = np.array(segments)

    return segments

In [ ]:
################################## Preparing local and central datasets
########## the local datasets the smarphone modality for each user will be stored as local_datasets_phone
########## the centralized datastes of phone for all users will be stored as x_central_phone & y_central_phone

OVERLAP = 0.2   # in precentage
Sample_FREQ = 50 # in Htz
seg_size = 2 #in second
clean_clients = [x for x in range(35) if x not in [0,2,5,6,17,27,30]]
local_datasets_phone = []
for client_idx in clean_clients:
    
    for class_id in range(9):
        phone_modal = client_set_phone[client_idx][class_id]
    #     phone_modal = zero_score(phone_modal)
        for sig in range(7):
            if sig == 0:
                x_data = segment_data_phone(phone_modal[sig],segment_size=seg_size, sampling_frequency=Sample_FREQ, overlap=OVERLAP)
                x_data = x_data[...,1:] #exluding the timestamps
                y_data = np.zeros([x_data.shape[0]]) + class_id

            else:
                segments = segment_data_phone(phone_modal[sig],segment_size=seg_size, sampling_frequency=Sample_FREQ, overlap=OVERLAP)
                x_data = np.concatenate([x_data,segments[...,1:]],axis=-1)
        if class_id == 0:
            x_dataset_phone = x_data
            y_dataset_phone = y_data
            if client_idx == clean_clients[0]:
                x_central_phone = x_dataset_phone
                y_central_phone = y_dataset_phone
        else:
            x_dataset_phone = np.concatenate([x_dataset_phone,x_data],axis=0)
            y_dataset_phone = np.concatenate([y_dataset_phone,y_data],axis=0)
    local_datasets_phone.append( [x_dataset_phone,y_dataset_phone] )
    x_central_phone = np.concatenate((x_central_phone,x_dataset_phone))
    y_central_phone = np.concatenate((y_central_phone,y_dataset_phone))

## 1-3 Speaker Segmentation

In [ ]:
import numpy as np

def segment_audio(data, segment_size=2, sampling_frequency=1000, overlap=0.5):
    # Segment the data
    segments = []
    step_size = int(segment_size * sampling_frequency * (1 - overlap))
    for i in range(0, len(data) - segment_size * sampling_frequency + 1, step_size):
        segment = data[i:i + segment_size * sampling_frequency]
        segments.append(segment)

    # Convert list of segments to numpy array
    segments = np.array(segments)

    return segments

In [ ]:
OVERLAP = 0.20   # in precentage
Sample_FREQ = sampling_rate # in Hz
seg_size = 2 #in second
def create_dataset(client_set,client_id, sample_freq, segment_size, overlap):
    for class_id in range(9):
        speaker_modal = client_set[client_id][class_id]
    #     phone_modal = zero_score(phone_modal)
        for sig in range(7):
            if sig == 0:
                x_data = segment_audio(speaker_modal[sig],segment_size=seg_size, sampling_frequency=Sample_FREQ, overlap=OVERLAP)
                seg_end = x_data.shape[1]-1
                wind_end = x_data.shape[0]-1
#                 x_data = x_data[...,1:] #exluding the timestamps
                y_data = np.zeros([wind_end]) + class_id

            else:
                segments = segment_audio(speaker_modal[sig],segment_size=seg_size, sampling_frequency=Sample_FREQ, overlap=OVERLAP)
                x_data = np.concatenate((x_data[:wind_end,:seg_end],segments[:wind_end,:seg_end]),axis=-1)
        if class_id == 0:
            x_dataset = x_data
            y_dataset = y_data
        else:
            x_dataset = np.concatenate([x_dataset,x_data],axis=0)
            y_dataset = np.concatenate([y_dataset,y_data],axis=0)
    return x_dataset, y_dataset

In [ ]:
X_data = []
Y_data = []
for id in range(35):
    if id == 31-1 or id==1-1 or id ==7-1 or id ==28-1 or id ==3-1 or id ==6-1 or id==18-1:
        continue
    x_dataset, y_dataset = create_dataset(client_set,id, Sample_FREQ, seg_size, OVERLAP)
    X_data.append(x_dataset)
    Y_data.append(y_dataset)
    
print(y_dataset.shape)
print(x_dataset.shape) ## (data sample, sequence of the signal, audio)

In [ ]:
import numpy as np
import librosa

def calculate_features(signal, sampling_rate=16000, n_mfcc=13):
    # Calculate the MFCCs for the signal
    mfccs = librosa.feature.mfcc(y=signal, sr=sampling_rate, n_mfcc=n_mfcc)
    
#     Calculate the Chroma feature
    chroma = librosa.feature.chroma_stft(y=signal, sr=sampling_rate)
    
    # Calculate the Spectral Contrast
#     spec_contrast = librosa.feature.spectral_contrast(y=signal, sr=sampling_rate)
    
    # Calculate the Tonnetz
#     tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(signal), sr=sampling_rate)
    
    # Concatenate the features into one array
    features = np.concatenate([mfccs, chroma], axis=0)
#     features = mfccs
    return features

# Your dataset
x_dataset = X_data[0]

# New dataset

New_X_data = []
for client in range(len(X_data)):
    new_x_dataset = []
    # Iterate over the samples in the dataset
    for sample in X_data[client]:
        # Calculate the features for each signal
        new_sample = calculate_features(sample,sampling_rate = sampling_rate)

        # Add the new sample to the new dataset
        new_x_dataset.append(new_sample)

    # Now new_x_dataset is a list of the same structure as x_dataset, but with the audio signals replaced by their features
    new_x_dataset = np.array(new_x_dataset)
    New_X_data.append(new_x_dataset)
    print(f"extracting features for client {client}")

In [ ]:
local_datasets_audio_x = New_X_data
local_datasets_audio_y = Y_data 
N = 0
for i in range(28):
    N = N + len(local_datasets_audio_y[i])

for i in range(28):
    if i == 0:
        x_centralized_audio = local_datasets_audio_x[i]
        y_centralized_audio = local_datasets_audio_y[i]

    else:
        x_centralized_audio = np.concatenate([x_centralized_audio,local_datasets_audio_x[i]])
        y_centralized_audio = np.concatenate([y_centralized_audio,local_datasets_audio_y[i]])

In [ ]:
local_datasets_audio = list()
for i in range(28):
    local_datasets_audio.append([New_X_data[i], Y_data[i]])

## 3- Aligning datasets

In [ ]:
## Aligning test data from phone and watch
aligned_data = list()
local_paired_watch = []
local_paired_phone = []
local_paired_audio = []
local_paired_label = []
for client in range(28):
    for i in range(9):
        indices_phone = np.where(np.array(local_datasets_phone[client][1]) == i)[0]
        indices_watch = np.where(np.array(local_datasets_watch[client][1]) == i)[0]
        indices_audio = np.where(np.array(local_datasets_audio[client][1]) == i)[0]
        min_len = np.min([len(indices_phone),len(indices_watch),len(indices_audio)])
        if i == 0:
            paired_label = local_datasets_watch[client][1][indices_watch[0:min_len]]
            
            paired_watch = local_datasets_watch[client][0][indices_watch[0:min_len]]
            paired_phone = local_datasets_phone[client][0][indices_phone[0:min_len]]
            paired_audio = local_datasets_audio[client][0][indices_audio[0:min_len]]
        else:
            paired_label = np.concatenate([paired_label,local_datasets_watch[client][1][indices_watch[0:min_len]]])
            
            paired_watch = np.concatenate([paired_watch,local_datasets_watch[client][0][indices_watch[0:min_len]]])
            paired_phone = np.concatenate([paired_phone,local_datasets_phone[client][0][indices_phone[0:min_len]]])
            paired_audio = np.concatenate([paired_audio,local_datasets_audio[client][0][indices_audio[0:min_len]]])
        for j in range(min_len):
            aligned_watch_data = local_datasets_watch[client][0][indices_watch[j]]
            aligned_phone_data = local_datasets_phone[client][0][indices_phone[j]]
            aligned_audio_data = local_datasets_audio[client][0][indices_audio[j]]
            aligned_labels = local_datasets_phone[client][1][indices_phone[j]]
            aligned_data.append([aligned_phone_data,aligned_watch_data,aligned_audio_data,aligned_labels])
    
    local_paired_label.append(paired_label)
    
    local_paired_watch.append(paired_watch)
    local_paired_phone.append(paired_phone)
    local_paired_audio.append(paired_audio)

print(len(aligned_data))

In [ ]:
###### Shuffle
for i in range(28):
    combined_data = list(zip(local_paired_watch[i],local_paired_phone[i], local_paired_audio[i], local_paired_label[i]))
    # Shuffle the combined data
    np.random.shuffle(combined_data)
    # Unzip the shuffled data
    local_paired_watch[i], local_paired_phone[i], local_paired_audio[i],local_paired_label[i] = zip(*combined_data)
